# Realtime inference with AWS DeepLens

This accompanies my article on Medium. I don't know the final URL of that draft yet, but you should be able to find it via https://medium.com/@tomgrek.

To run this you will need to grab the pretrained model:
```
wget https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz
tar -xvzf inception_v3_2016_08_28_frozen.pb.tar.gz
```

In [1]:
import tensorflow as tf

In [2]:
graph = tf.Graph()
graph_def = tf.GraphDef()
with open('./inception_v3_2016_08_28_frozen.pb', "rb") as f:
    graph_def.ParseFromString(f.read())
with graph.as_default():
    tf.import_graph_def(graph_def)

In [9]:
def load_labels(label_file):
  label = []
  proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
  for l in proto_as_ascii_lines:
    label.append(l.rstrip())
  return label

In [10]:
input_layer = "input"
output_layer = "InceptionV3/Predictions/Reshape_1"

In [11]:
input_name = "import/" + input_layer
output_name = "import/" + output_layer
input_operation = graph.get_operation_by_name(input_name)
output_operation = graph.get_operation_by_name(output_name)

In [13]:
import numpy as np
import awscam
import cv2
import time

In [36]:
for i in range(0,10):
    got, frame = awscam.getLastFrame()
    cv2.imwrite('output.jpg', frame)
    image_reader = tf.image.decode_jpeg(tf.read_file('./output.jpg', "file_reader"), channels=3, name="jpeg_reader")
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(dims_expander, [299, 299])
    normalized = tf.divide(tf.subtract(resized, [0]), [255])
    sess = tf.Session()
    image = sess.run(normalized)
    with tf.Session(graph=graph) as sess:
        results = sess.run(output_operation.outputs[0], {
            input_operation.outputs[0]: image
        })
    results = np.squeeze(results)
    top_k = results.argsort()[-5:][::-1]
    labels = load_labels("imagenet_slim_labels.txt")
    for i in top_k:
        print(labels[i], results[i])
    time.sleep(2)

('monitor', 0.28125063)
('screen', 0.15084484)
('television', 0.12388436)
('water jug', 0.09155932)
('beer glass', 0.04396197)
('monitor', 0.22072905)
('water jug', 0.14242181)
('television', 0.12632273)
('screen', 0.10927667)
('beer glass', 0.066972956)
('monitor', 0.27544972)
('water jug', 0.12971021)
('screen', 0.12900297)
('television', 0.101049915)
('beer glass', 0.059571203)
('monitor', 0.21874452)
('water jug', 0.15469997)
('screen', 0.12310624)
('television', 0.0992532)
('beer glass', 0.060115915)
('monitor', 0.29284143)
('screen', 0.14606945)
('television', 0.11821335)
('beer glass', 0.073011056)
('water jug', 0.036694065)
('running shoe', 0.078758344)
('hand blower', 0.05470746)
('sandal', 0.036435798)
('electric fan', 0.033989843)
('bassinet', 0.019181341)
('crash helmet', 0.16109857)
('seat belt', 0.10279817)
('bucket', 0.06464464)
('toilet seat', 0.04851903)
('oxygen mask', 0.025366016)
('toilet seat', 0.17103137)
('crash helmet', 0.07273375)
('toilet tissue', 0.06422536)
